In [1]:
import glob
import seaborn as sns
import pandas as pd
import matplotlib.image as img
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
import cv2
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix
from  sklearn.linear_model import LogisticRegression
import tensorflow as tf
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
desire_size = (90, 90)
labels = list(map(lambda x: x.split("\\")[1], glob.glob("data/train/*/*")))
IDS = list(map(lambda x: x.split("\\")[2], glob.glob("data/train/*/*")))
le = preprocessing.LabelEncoder()
le.fit(list(set(labels)))
num_out = len(le.classes_)
pd.Series(labels).value_counts()

Loose Silky-bent             620
Common Chickweed             566
Scentless Mayweed            480
Small-flowered Cranesbill    480
Fat Hen                      432
Sugar beet                   361
Charlock                     355
Cleavers                     263
Black-grass                  246
Shepherds Purse              233
Maize                        216
Name: count, dtype: int64

In [3]:
labels

['Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-

In [4]:
ls_size = []
X = []
y = []
for file in glob.glob("data/train/*/*"):
    label = file.split("\\")[1]
    id_img = file.split("\\")[2]
    # gray scale para tener los 3 colores quite la L
    im = Image.open(file).convert("L")
    ls_size.append([im.size[0],im.size[1],label,id_img])
    img_resize = im.resize(desire_size)
    X.append(np.array(img_resize).flatten())
    y.append(label)

In [5]:
# datos ready for CNN
y_img = to_categorical(le.transform(y), dtype ="uint8")
X_img = np.reshape(np.array(X), (-1, desire_size[0], desire_size[1]))
X_train,X_test, y_train,y_test = train_test_split(X_img,y_img,test_size=0.2)

In [6]:
X_oob = []
file_oob = []
for file in glob.glob("data/test/*"):
    im = Image.open(file).convert('L')
    img_resize = im.resize(desire_size)
    X_oob.append(np.array(img_resize).flatten())
    file_oob.append(file.split("\\")[1])

In [7]:
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=8, input_shape=(desire_size[0],desire_size[1],1),
                 kernel_size=(4,4),padding='same', activation='relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=(2,2), strides=(1,1), padding='same', activation='relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=(2,2), strides=(1,1), padding='same', activation='relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1), padding='same', activation='relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(200, activation='relu'))

# Add Dropout
model.add(Dropout(0.1))

# 2nd Fully Connected Layer
model.add(Dense(100, activation='relu'))

# Add Dropout
model.add(Dropout(0.1))

# 3rd Fully Connected Layer
model.add(Dense(50, activation='relu'))

# Add Dropout
model.add(Dropout(0.1))

# Output Layer
model.add(Dense(num_out, activation='softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 90, 90, 8)         136       
                                                                 
 max_pooling2d (MaxPooling2  (None, 44, 44, 8)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 22, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 32)        2080      
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 22, 64)        8

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=40,     # Rango de rotación
    width_shift_range=0.2, # Rango de traslación horizontal
    height_shift_range=0.2, # Rango de traslación vertical
    shear_range=0.2,       # Rango de deformación
    zoom_range=0.2,        # Rango de zoom
    horizontal_flip=True,  # Inversión horizontal
    fill_mode='nearest'    # Modo de relleno en caso de transformación
)

In [10]:
model.fit(X_train,y_train,  epochs=10, batch_size=268, validation_data=(X_test, y_test))

Epoch 1/10
 1/13 [=>............................] - ETA: 45s - loss: 3.8981 - accuracy: 0.1194

KeyboardInterrupt: 

In [ ]:
#oob = Out Of Bag
X_oob_img = np.reshape(np.array(X_oob), (-1, desire_size[0], desire_size[1]))
y_hat = le.inverse_transform(np.argmax(model.predict(X_oob_img), axis=1))
df_oob = pd.DataFrame(data= zip(file_oob, y_hat),
                      columns=["file","species"])
df_oob.to_csv("data/prediction_cnn.csv", index=False)